# Simulaion

In [1]:
import numpy as np
import time
from scipy.spatial.transform import Rotation as R
from Go2Py.sim.mujoco import Go2Sim
from Go2Py.robot.model import Go2Model
import mujoco

# Initiate the robot model 
model = Go2Model()
robot = Go2Sim(mode='lowlevel')
robot.standUpReset()

In [2]:
robot.model.opt.gravity[2] = 0

In [ ]:
while True:
    q = robot.getJointStates()["q"]
    print(q)
    time.sleep(1)


In [4]:
# Data to be stored
data = [] # should contain q, q_target, dq, tau_est, tau

In [5]:
import numpy as np

def generate_traj(A, f, t, current_foot_poses):
    """
    Generate the foot trajectories using a sine wave and modify the current foot positions.

    Args:
        A (float): Amplitude of the sine wave in centimeters.
        f (float): Frequency of the sine wave in Hz.
        t (float): Current time in seconds.
        current_foot_poses (dict): A dictionary where keys are frame names (e.g., 'FR_foot') 
                                   and values are 4x4 transformation matrices.

    Returns:
        np.ndarray: A numpy array of size 12 containing the modified foot positions 
                    in the world frame (flattened as [FR_x, FR_y, FR_z, FL_x, FL_y, FL_z, ...]).
    """
    A = A / 100  # Convert amplitude to meters
    omega = 2 * np.pi * f
    x_traj = A * np.cos(omega * t)  # X trajectory (circular motion)
    z_traj = A * np.sin(omega * t)  # Z trajectory (circular motion)
    y_traj = A * np.sin(2 * omega * t)  # Y trajectory (lateral motion)

    x_traj_off = A * np.cos(omega * t + np.pi)  # X trajectory (circular motion)
    z_traj_off = A * np.sin(omega * t + np.pi)  # Z trajectory (circular motion)
    y_traj_off = A * np.sin(2 * omega * t + np.pi) 

    # Offsets for each foot
    feet_offsets = {
        # 'FR_foot': np.array([x_traj_off, y_traj_off, z_traj_off]),
        'FL_foot': np.array([x_traj, y_traj, z_traj]),
        # 'RR_foot': np.array([x_traj, y_traj, z_traj]),  
        # 'RL_foot': np.array([x_traj_off, y_traj_off, z_traj_off]),
    }

    # Store the updated foot positions in a numpy array
    feet_pos = np.zeros(12)  # 4 feet × 3 coordinates (x, y, z)

    foot_order = ['FR_foot', 'FL_foot', 'RR_foot', 'RL_foot']  # Ensure correct order
    for i, foot in enumerate(foot_order):
        if foot in current_foot_poses:
            # Extract translation (position) from the homogeneous transformation matrix
            translation = current_foot_poses[foot][:3, 3]  
            # Apply the offset
            modified_translation = translation + feet_offsets[foot]
            # Store in the numpy array
            feet_pos[3 * i: 3 * i + 3] = modified_translation  

    return feet_pos  # Returns a 12-element numpy array

In [6]:
def getT():
    position, quaternion = robot.getPose()
    rotation_matrix = R.from_quat([quaternion[1], quaternion[2], quaternion[3], quaternion[0]]).as_matrix()
    T = np.eye(4)
    T[:3, :3] = rotation_matrix
    T[:3, 3] = position
    
    return T

In [ ]:
# Data Collector 
T = np.eye(4)
state = robot.getJointStates()
# model.updateAllPose(state['q'], state['dq'], T, np.zeros(6))

start_time = time.time()
q_nominal = state['q']
# nominal_foot_poses = model.forwardKinematics(T, q_nominal) # Get nominal foot poses from forward kinematics
nominal_foot_poses = []

As = np.arange(4).tolist()
freqs = np.arange(1).tolist()

for f in freqs:
    omega = 2 * np.pi * f
    for A in As:
        print("current Frequancy:", f,"and Amplitude:", A)
        start_time = time.time()
        # For each combination of A, f collect data for 10 seconds
        while (time.time() - start_time < 10):
            
            # Get current state and Transformation matrix
            state = robot.getJointStates()
            T = getT()
            
            # Update Poses
            # model.updateAllPose(state['q'], state['dq'], T, np.zeros(6))
            
            # Generate foot traj
            t = time.time() - start_time
            feet_pos = generate_traj(A, f, t, nominal_foot_poses) 
            # Apply inverse kinematics to get q_target
            q_target = model.inverseKinematics(getT(),feet_pos) 
            # Apply q_target
            dq = np.zeros(12)
            kp = np.ones(12)*3.0
            kd = np.ones(12)*0.1
            kp[:3] = 0
            kp[6:] = 0
            kd[:3] = 0
            kd[6:] = 0
            print(kp, kd)
            pd_law = 1*6.3*(q_target-state['q']) + 1*0.5*(dq-state['dq'])
            tau_ff = pd_law

            # robot.setCommands(q_target, dq, kp, kd, np.zeros(12))
            robot.step()
            if np.isnan(state['tau_est'].reshape(-1)).any():
                print("Warning: NaN values detected in tau_est")
            # Save data
            data.append([state['tau_est'].reshape(-1), np.zeros(12), state['q'], q_target, state['dq']])
            time.sleep(0.01)

In [ ]:
# Verify dimensions
print("tau:", tau_ff.shape)
print("tau_est", state['tau_est'].shape)
print("q:", state['q'].shape)
print("q_target", q_target.shape)
print("dq:", dq.shape)


data = np.array(data)
# Save to pickle file
import pickle
with open('actuator_net_dataset_sim2.pkl', 'wb') as f:
    pickle.dump(
        {'tau_est': data[:,0],
         'torques': data[:,1],
         'joint_pos': data[:,2],
         'joint_pos_target': data[:,3],
         'joint_vel': data[:,4]},
         f
    )

# Real Robot

In [1]:
from Go2Py.robot.interface import GO2Real
from Go2Py.robot.model import Go2Model
from scipy.spatial.transform import Rotation as R
import numpy as np
import time


model = Go2Model()
robot = GO2Real()
data =[]

pygame 2.6.1 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
state = robot.getJointStates()
print(state.keys())
print(state)

data = [] # should contain q, q_target, dq, tau_est, tau

dict_keys(['q', 'dq', 'tau_est', 'temperature'])
{'q': [-0.04112428426742554, 1.2605890035629272, -2.787384033203125, 0.045272305607795715, 1.2619211673736572, -2.8102264404296875, -0.3544110059738159, 1.2830191850662231, -2.8211264610290527, 0.34193670749664307, 1.2835944890975952, -2.7998480796813965], 'dq': [0.027128666639328003, -0.011626571416854858, -0.0020220123697072268, -0.01937761902809143, 0.03100419044494629, -0.03033018670976162, -0.054257333278656006, -0.10463915020227432, 0.036396224051713943, -0.03875523805618286, 0.015502095222473145, 0.006066037341952324], 'tau_est': [-0.049476563930511475, 0.049476563930511475, -0.04741504043340683, -0.07421484589576721, -0.07421484589576721, 0.04741504043340683, 0.049476563930511475, 0.049476563930511475, 0.0, 0.024738281965255737, 0.0, 0.0], 'temperature': [26.0, 26.0, 26.0, 25.0, 25.0, 26.0, 26.0, 26.0, 26.0, 26.0, 25.0, 26.0]}


In [4]:
for i in range(1000000):
    q = np.zeros(12) 
    dq = np.zeros(12)
    kp = np.ones(12)*0.0
    kd = np.ones(12)*0.0
    tau = np.zeros(12)
    tau[0] = 0.0
    robot.setCommands(q, dq, kp, kd, tau)
    time.sleep(0.01)    

KeyboardInterrupt: 

In [6]:
import numpy as np

def generate_traj(A, f, t, current_foot_poses):
    """
    Generate the foot trajectories using a sine wave and modify the current foot positions.

    Args:
        A (float): Amplitude of the sine wave in centimeters.
        f (float): Frequency of the sine wave in Hz.
        t (float): Current time in seconds.
        current_foot_poses (dict): A dictionary where keys are frame names (e.g., 'FR_foot') 
                                   and values are 4x4 transformation matrices.

    Returns:
        np.ndarray: A numpy array of size 12 containing the modified foot positions 
                    in the world frame (flattened as [FR_x, FR_y, FR_z, FL_x, FL_y, FL_z, ...]).
    """
    A = A / 100  # Convert amplitude to meters
    omega = 2 * np.pi * f
    x_traj = A * np.cos(omega * t)  # X trajectory (circular motion)
    z_traj = A * np.sin(omega * t)  # Z trajectory (circular motion)
    y_traj = A * np.sin(2 * omega * t)  # Y trajectory (lateral motion)

    x_traj_off = A * np.cos(omega * t + np.pi)  # X trajectory (circular motion)
    z_traj_off = A * np.sin(omega * t + np.pi)  # Z trajectory (circular motion)
    y_traj_off = A * np.sin(2 * omega * t + np.pi) 

    # Offsets for each foot
    feet_offsets = {
        # 'FR_foot': np.array([x_traj_off, y_traj_off, z_traj_off]),
        'FR_foot': np.array([0, 0, 0]),
        'FL_foot': np.array([0, 0, z_traj]),
        'RR_foot': np.array([0, 0, 0]),  
        'RL_foot': np.array([0, 0, 0]),
        # 'RR_foot': np.array([x_traj, y_traj, z_traj]),  
        # 'RL_foot': np.array([x_traj_off, y_traj_off, z_traj_off]),
    }

    # Store the updated foot positions in a numpy array
    feet_pos = np.zeros(12)  # 4 feet × 3 coordinates (x, y, z)

    foot_order = ['FR_foot', 'FL_foot', 'RR_foot', 'RL_foot']  # Ensure correct order
    for i, foot in enumerate(foot_order):
        if foot in current_foot_poses:
            # Extract translation (position) from the homogeneous transformation matrix
            translation = current_foot_poses[foot][:3, 3]  
            # Apply the offset
            modified_translation = translation #+ feet_offsets[foot]
            # Store in the numpy array
            feet_pos[3 * i: 3 * i + 3] = modified_translation  

    return feet_pos  # Returns a 12-element numpy array

In [7]:
def getT():
    position, quaternion = robot.getPose()
    rotation_matrix = R.from_quat([quaternion[1], quaternion[2], quaternion[3], quaternion[0]]).as_matrix()
    T = np.eye(4)
    T[:3, :3] = rotation_matrix
    T[:3, 3] = position
    
    return T

In [ ]:
# get nominal foot positions
while True:

    q = np.zeros(12) 
    dq = np.zeros(12)
    kp = np.ones(12)*0.0
    kd = np.ones(12)*0.0
    tau = np.zeros(12)
    tau[0] = 0.0
    robot.setCommands(q, dq, kp, kd, tau)

    state = robot.getJointStates()
    q = np.array(state['q'])
    foot_pos = model.forwardKinematics(np.eye(4), q) # Get nominal foot poses from forward kinematics
    print(foot_pos['FL_foot'][:3,3])

    
    time.sleep(1)

#  [ 0.34835959  0.22471502 -0.07046597]
#  [ 0.20625531  0.2808269  -0.16363015]
#  [ 0.04853118  0.21627886 -0.1713757 ]


In [9]:
# Data Collector 
T = np.eye(4)
state = robot.getJointStates()
# model.updateAllPose(state['q'], state['dq'], T, np.zeros(6))

start_time = time.time()
q_nominal = np.array(state['q'])
nominal_foot_poses = model.forwardKinematics(T, q_nominal) # Get nominal foot poses from forward kinematics
FL_foot_poses = [[0.34835959,  0.22471502, -0.15046597], [ 0.20625531,  0.2808269 , -0.16363015], [ 0.04853118,  0.21627886 ,-0.1713757 ]]
# FL_foot_poses  = [[ 0.20625531,  0.2808269 , -0.19363015]]
# #As = list(range(2, 7))
# As = np.arange(4).tolist()
# freqs = np.arange(1).tolist()


# for i in range(len(FL_foot_poses)):
#     for f in freqs:
#         omega = 2 * np.pi * f
#         for A in As:
#             print("current Frequancy:", f,"and Amplitude:", A)
#             start_time = time.time()
#             # For each combination of A, f collect data for 10 seconds
#             while (time.time() - start_time < 10):
                
#                 # Get current state and Transformation matrix
#                 state = robot.getJointStates()
#                 T = np.eye(4)
                
#                 # Update Poses
#                 # model.updateAllPose(state['q'], state['dq'], T, np.zeros(6))
                
#                 # Generate foot traj
#                 t = time.time() - start_time
#                 nominal_foot_poses['FL_foot'][:3,3] = FL_foot_poses[i]
#                 feet_pos = generate_traj(A, f, t, nominal_foot_poses) 
#                 # Apply inverse kinematics to get q_target
#                 q_target = model.inverseKinematics(T,feet_pos) 
#                 # Apply q_target
#                 dq = np.zeros(12)
#                 kp = np.ones(12)*3.0
#                 kd = np.ones(12)*0.1
#                 kp[:3] = 0
#                 kp[6:] = 0
#                 kd[:3] = 0
#                 kd[6:] = 0
#                 pd_law = 1*6.3*(q_target-state['q']) + 1*0.5*(dq-state['dq'])
#                 tau_ff = pd_law

#                 robot.setCommands(q_target, dq, kp, kd, tau_ff)

#                 tau_est = np.array(state['tau_est'])  # Convert list to NumPy array
#                 if np.isnan(tau_est.reshape(-1)).any():
#                     print("Warning: NaN values detected in tau_est")
#                 # Save data
#                 data.append([tau_est, tau_ff, state['q'], q_target, state['dq']])
#                 time.sleep(0.01)
As = np.arange(0.4, 0.45, 0.025).tolist()
freqs = np.arange(1, 3, 1).tolist()

for i in range(len(FL_foot_poses)):
    for f in freqs:
        omega = 2 * np.pi * f
        for A in As:
            print("current Frequancy:", f,"and Amplitude:", A)
            start_time = time.time()
            # For each combination of A, f collect data for 10 seconds
            while (time.time() - start_time < 10):
                
                # Get current state and Transformation matrix
                state = robot.getJointStates()
                T = np.eye(4)
        
                # Generate foot traj
                t = time.time() - start_time
                nominal_foot_poses['FL_foot'][:3,3] = FL_foot_poses[i]
                feet_pos = generate_traj(A, f, t, nominal_foot_poses) 
                # Apply inverse kinematics to get q_target
                q_target = model.inverseKinematics(T,feet_pos) 
                q_target[5] = q_target[5] + A * np.sin(omega*t)
                # Apply q_target
                dq = np.zeros(12)
                kp = np.ones(12)*3.0
                kd = np.ones(12)*0.1
                tau_ff = np.zeros(12)
                kp[:3] = 0 
                kp[6:] = 0 
                kd[:3] = 0 
                kd[6:] = 0 
                # Calculate the idealized torques for plotting purposes
                torques = kp*(q_target - state['q']) + kd*(dq - state['dq'])

                robot.setCommands(q_target, dq, kp, kd, tau_ff)
                tau_est = np.array(state['tau_est'])
                if np.isnan(tau_est.reshape(-1)).any():
                    print("Warning: NaN values detected in tau_est")
                # Save data
                data.append([tau_est.reshape(-1), torques, state['q'], q_target, state['dq']])
                time.sleep(0.01)

current Frequancy: 1 and Amplitude: 0.4
current Frequancy: 1 and Amplitude: 0.42500000000000004
current Frequancy: 2 and Amplitude: 0.4
current Frequancy: 2 and Amplitude: 0.42500000000000004
current Frequancy: 1 and Amplitude: 0.4
current Frequancy: 1 and Amplitude: 0.42500000000000004
current Frequancy: 2 and Amplitude: 0.4
current Frequancy: 2 and Amplitude: 0.42500000000000004
current Frequancy: 1 and Amplitude: 0.4
current Frequancy: 1 and Amplitude: 0.42500000000000004
current Frequancy: 2 and Amplitude: 0.4
current Frequancy: 2 and Amplitude: 0.42500000000000004


In [10]:
# Verify dimensions
tau_est = np.array(state['tau_est'])
q = np.array(state['q'])
print("tau:", tau_ff.shape)
print("tau_est", tau_est.shape)
print("q:", q.shape)
print("q_target", q_target.shape)
print("dq:", dq.shape)


data = np.array(data)
# Save to pickle file
import pickle
with open('actuator_net_dataset_real3.pkl', 'wb') as f:
    pickle.dump(
        {'tau_est': data[:,0],
         'torques': data[:,1],
         'joint_pos': data[:,2],
         'joint_pos_target': data[:,3],
         'joint_vel': data[:,4]},
         f
    )

tau: (12,)
tau_est (12,)
q: (12,)
q_target (12,)
dq: (12,)


1747852173.638201 [0]        tev: ddsi_udp_conn_write to udp/239.255.0.1:7400 failed with retcode -1
1747852173.638252 [0]        tev: ddsi_udp_conn_write to udp/192.168.123.18:7410 failed with retcode -1
1747852174.438372 [0]        tev: ddsi_udp_conn_write to udp/239.255.0.1:7400 failed with retcode -1
1747852174.438411 [0]        tev: ddsi_udp_conn_write to udp/192.168.123.18:7410 failed with retcode -1
1747852176.038494 [0]        tev: ddsi_udp_conn_write to udp/192.168.123.18:7410 failed with retcode -1
1747852176.038529 [0]        tev: ddsi_udp_conn_write to udp/192.168.123.18:47883 failed with retcode -1
1747852177.638607 [0]        tev: ddsi_udp_conn_write to udp/192.168.123.18:7410 failed with retcode -1
1747852179.036764 [0]        tev: ddsi_udp_conn_write to udp/192.168.123.18:7410 failed with retcode -1
1747852179.136244 [0]        tev: ddsi_udp_conn_write to udp/239.255.0.1:7400 failed with retcode -1
1747852179.136282 [0]        tev: ddsi_udp_conn_write to udp/192.168.123